<a href="https://colab.research.google.com/github/oraziotorre/TTinsight/blob/main/TTinsight_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TTinsight


---


## Data Cleaning & Feature Engineering

In questo notebook vedremo tutto il processo per il caricamento, l'analisi e la pulizia dei dati. Sceglieremo poi le feature migliori e generaremo il dataset finale

Vogliamo addrestrare un modello che predica la probabilità di "W" nella colonna matcht result

# **Inizializzazione**

In [15]:
# Pandas per la gestione dei dati
import pandas as pd

# Gestione per One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [16]:
dataset = pd.read_csv("raw_dataset.csv")

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ..."
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ..."
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ..."
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ..."
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ..."
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ..."
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ..."
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]"


# **Etichettatura**

Il dataset avrà una nuova colonna ```set_result``` che conterrà il risultato del set per ogni partita:

- **1** per la classe **Vincitori** del set
- **0** per la classe **Sconfitti** del set

Questo tipo di etichettatura è utile per i nostri LSTM e Regressione Logistica, dove l'obiettivo sarà quello di prevedere se un giocatore vincerà o perderà un set sulla base di altre informazioni.


In [17]:
# Funzione per convertire i punteggi da stringa a lista di interi
def convert_to_int_list(points_str):
    # Verifica che la stringa sia valida
    if isinstance(points_str, str):
        # Rimuove le parentesi quadre e suddivide la stringa per ottenere i numeri
        return list(map(int, points_str.strip('[]').split(', ')))
    else:
        return None


# Funzione per determinare il risultato del set
def calculate_set_result(player_points, opponent_points):
    if player_points:
        if player_points[-1] > opponent_points[-1]:
            return 1  # Vittoria per il giocatore
        else:
            return 0  # Sconfitta per il giocatore
    else:
        return -1  # Dati non conformi

# Inizializzo una lista vuota per la nuova colonna con le etichette
set_results = []

# Trasformo i punteggi in liste di interi utilizzando la funzione definita
dataset['player_points'] = dataset['player_points'].apply(convert_to_int_list)
dataset['opponent_points'] = dataset['opponent_points'].apply(convert_to_int_list)

# Itero su ogni riga e dai punteggi definisce l'etichetta
for index, row in dataset.iterrows():
    result = calculate_set_result(row['player_points'], row['opponent_points'])
    set_results.append(result)

# Aggiungo la nuova colonna 'set_result' al DataFrame
dataset['set_result'] = set_results

# Visualizzo il risultato
dataset




,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


# **Analisi dei dati**


## Controllo del bilanciamento dei dati
Una delle prime cose che possiamo verificare è se il dataset sia **bilanciato**. Per cui, controlliamo il numero di istanze **1** e **0** della colonna *set_result*

In [18]:
# Numero di elementi per la classe "Vincitori"
print(len(dataset[(dataset['set_result'] == 1)]))
# Numero di elementi per la classe "Sconfitti"
print(len(dataset[(dataset['set_result'] == 0)]))

# Totale
print(len(dataset))


69607
69607
144194


Notiamo che abbiamo un dataset non troppo sbilanciato

## **Analisi delle caratteristiche e delle distribuzioni dei dati**
Usiamo `describe()` per avere una overview sulle distribuzioni dei dati numerici

In [19]:
# Verifico la correttezza dei campi numerici
dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,144194.000000,144194.000000,144194.000000,144194.000000,35116.000000,144194.000000,35116.000000,144194.000000,144194.000000,144194.000000,144194.000000
mean,2702.487149,1556.146095,2058.918221,123836.818280,125800.822332,123836.818280,125800.822332,2.086800,2.086800,3.146705,0.448195
std,156.933844,5453.961129,1575.253127,17169.115984,17804.203785,17169.115984,17804.203785,1.250522,1.250522,0.353970,0.562487
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,2.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132081.000000,133124.000000,132081.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,5.000000,5.000000,5.000000,1.000000


Notiamo che nella colonna "set_required_to_win" sono presenti dei valori rari non nel nostro interesse (2 e 5)

## **Controllo dei valori nulli**

Verifico i valori mancanti nei dati

In [20]:
# Identificare i valori NaN nel dataset
nan_mask = dataset.isna()

# Conta i valori NaN (valori mancanti) per ciascuna colonna del dataset
nan_count = nan_mask.sum()

# Verifico i valori null del raw_dataset
nan_count


,0
event_id,0
match_id,0
match_format,0
players_gender,0
match_stage,0
stage_id,0
match_duration,0
match_start_time,0
player_id,0
player_2_id,109078


#**Operazione sulle feature**


Facciamo alcune considerazioni sulle caratteristiche del nostro dataset e identifichiamo possibili **miglioramenti** per ottimizzarlo in vista del prossimo training del modello:

* Possibile **trasformare i formati T** nella colonna ```match_format``` in formati che siano o S o D

* La colonna ```match_stage``` può essere modificata per attribuire **maggiore importanza alle partite finali** (FNL) rispetto a tutte le altre, migliorando il peso di queste informazioni nel training del modello.

* Le colonne ```sets_required_to_win``` e ```current_match_state``` fanno entrambe riferimento al numero di set necessari per vincere una partita. Per rendere il dataset più chiaro e organizzato, possiamo **sostituirle** con due nuove colonne: ```sets_to_win``` e ```opponent_sets_to_win```, che rappresentano rispettivamente il numero di set necessari al giocatore e all'avversario per vincere la partita.

* Per la colonna ```players_gender``` si può semplificare il formato tramite l'operazione di ***One-Hot Encoding*** nel caso di dataset_all_matches o tramite una semplice codifica binaria nel caso del dataset_single_matches

*  Le colonne ```match_format```, ```player_id```, ```player_2_id``` **non sono necessarie** per il training del modello e possono essere cancellate




## **Colonna *match_format***

Modifica la colonna *match_format*, sostituendo tutti i valori ' T ' (partita di team) con ' S ' (partita di singolo) o ' D ' (partita di doppio)

Viene eseguita questa operazione per **evitare la rimozione di valori** che potrebbero ritornarci utile successivamente nello sviluppo del nostro modello

In [21]:
# Funzione per determinare il match_format quando è Team (Singolo o Doppio)
def determine_match_format(player_2_id):
    # Controlla se il valore di player_2_id è NaN (valore mancante)
    if pd.isna(player_2_id):
        return 'S'  # Ritorna 'S' per indicare una partita singola
    else:
        return 'D'  # Ritorna 'D' per indicare una partita doppia

# Applica la funzione determine_match_format alla colonna 'player_2_id' del dataset
# Modificando il valore nella colonna match_format di tipo S o D
dataset['match_format'] = dataset['player_2_id'].apply(determine_match_format)

# Mostra il dataset aggiornato
dataset


,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


In base a ciò che abbiamo appreso precedentemente da `describe()`, rimuoviamo i valori non desiderati della colonna *sets_required_to_win* uguali a 2 o 5.

Questi valori rari potrebbero rappresentare **formati di gioco non standard** (es. partite amichevoli o partite abbreviate per motivi eccezionali), non rilevanti per l'analisi



In [22]:
# Rimuoviamo le righe contenenti set con punteggi rari che non vogliamo considerare
dataset = dataset[~dataset['sets_required_to_win'].isin([2, 5])]

dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,144186.000000,144186.000000,144186.000000,144186.000000,35116.000000,144186.000000,35116.000000,144186.000000,144186.000000,144186.000000,144186.000000
mean,2702.497746,1556.146436,2058.904013,123837.064202,125800.822332,123837.064202,125800.822332,2.086791,2.086791,3.146686,0.448213
std,156.931733,5454.110892,1575.282189,17169.454336,17804.203785,17169.454336,17804.203785,1.250485,1.250485,0.353793,0.562468
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,3.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132081.000000,133124.000000,132081.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,4.000000,4.000000,4.000000,1.000000


Eliminiamo i valori nulli della colonna *player_points*

In [23]:
# Rimuoviamo tutte le righe del dataset in cui la colonna 'player_points' contiene valori NaN
dataset = dataset.dropna(subset=["player_points"])

# Verifichiamo che non esistano più righe con valori NaN nella colonna 'player_points'
dataset[dataset['player_points'].isnull()]

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result


##**Colonna *match_stage***

Al posto della colonna *match_stage* inseriamo *is_final* in modo da semplificarne i valori in:
- **FNL = 1**
- **Tutti gli altri stage del torneo = 0**

In questo modo possiamo dare maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, **migliorando il peso di queste informazioni** nel training del modello.

In [24]:
# Creiamo una nuova colonna 'is_final' che vale 1 se 'match_stage' è 'FNL' (finale), altrimenti 0
dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)

# Rimuoviamo la colonna 'match_stage' dal dataset poiché non è più necessaria
dataset = dataset.drop(columns=['match_stage'])

# Troviamo la posizione della colonna 'players_gender' nel dataset
players_gender_index = dataset.columns.get_loc('players_gender')

# Estraiamo la colonna 'is_final' e la inseriamo subito dopo 'players_gender'
is_final_column = dataset.pop('is_final')
dataset.insert(players_gender_index + 1, 'is_final', is_final_column)

# Mostra il dataset aggiornato
dataset


<ipython-input-24-a800c8080596>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)


,event_id,match_id,match_format,players_gender,is_final,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


##**Colonne *sets_required_to_win* e *current_match_state***

Il codice calcola il **numero di set mancanti alla vittoria** per un giocatore e per il suo avversario, basandosi su informazioni presenti nel dataset e creando due nuove colonne ```final_set``` e ```opponent_final_set```

Determiniamo se un giocatore o il suo avversario è a un set dalla vittoria
- Se il giocatore è un set dalla vittoria poniamo **final_set pari a 1**
- Se l'avversario è un set dalla vittoria poniamo **opponent_final_set pari a 1**
- Altrimenti sarà **pari a 0**

Elimina le colonne originali ```sets_required_to_win``` e ```current_match_state```, ormai non più necessarie.

Le colonne originali contenevano informazioni in un formato più complesso:
- *current_match_state* era una stringa del tipo **"x-y"**, che richiede parsing per separare i valori x e y.
- *sets_required_to_win* da sola **non fornisce** un'indicazione diretta del progresso del match.
Le nuove colonne forniscono le stesse informazioni in modo più esplicito, eliminando la necessità di operazioni intermedie (come lo split delle stringhe).

In [25]:
# Calcola la differenza tra i set richiesti per vincere e i set vinti dal giocatore e dall'avversario
final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[0].astype(int)
opponent_final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[1].astype(int)

# Crea una colonna 'final_set' che vale 1 se il giocatore è a un set dalla vittoria, altrimenti 0
dataset['final_set'] = (final_set_diff == 1).astype(int)

# Crea una colonna 'opponent_final_set' che vale 1 se l'avversario è a un set dalla vittoria, altrimenti 0
dataset['opponent_final_set'] = (opponent_final_set_diff == 1).astype(int)

# La posizione della colonna is_final viene determinata con get_loc.
is_final_index = dataset.columns.get_loc('is_final')

# Le nuove colonne final_set e opponent_final_set vengono temporaneamente rimosse (pop) e poi reinserite subito dopo la colonna is_final usando insert.
final_set_column = dataset.pop('final_set')
opponent_final_set_column = dataset.pop('opponent_final_set')

# Riposiziona le colonne 'final_set' e 'opponent_final_set' subito dopo 'is_final'
dataset.insert(is_final_index + 1, 'final_set', final_set_column)
dataset.insert(is_final_index + 2, 'opponent_final_set', opponent_final_set_column)

# Elimina le colonne originali 'sets_required_to_win' e 'current_match_state'
dataset.drop(['sets_required_to_win', 'current_match_state'], axis=1, inplace=True)

# Mostra il dataset aggiornato
dataset


,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


## **Colonna *players_gender***

Questa colonna verrà utilizzata per il **one-hot encoding**, che trasformerà le seguenti combinazioni di genere in vettori:

- **M** → `[1, 0, 0]` (Maschile)
- **W** → `[0, 1, 0]` (Femminile)
- **X** → `[0, 0, 1]` (Non specificato)

Il One-Hot-Encoding presenta dei vantaggi:
- **Evita ordini impliciti**: è preferibile a una codifica numerica diretta (es. M=1, W=2, X=3), perché quest'ultima potrebbe introdurre una relazione inesistente tra i valori.
- **Modelli di Machine Learning**: il modello di *regressione logistica* richiede input numerici. La trasformazione da categorie a numeri binari rende i dati compatibili.

In [26]:
# Importiamo OneHotEncoder per trasformare valori categoriali in colonne binarie
# sparse_output=False per ottenere una matrice densa anziché sparsa
encoder = OneHotEncoder(sparse_output=False)

# Applichiamo il One Hot Encoding alla colonna 'players_gender'
X_encoded = encoder.fit_transform(dataset[['players_gender']])

# Otteniamo i nomi delle nuove colonne dal One Hot Encoder
encoded_columns = encoder.get_feature_names_out(['players_gender'])

# Aggiungiamo le nuove colonne codificate al dataset
dataset[encoded_columns] = X_encoded  # Inseriamo la matrice codificata come nuove colonne nel dataset
dataset[encoded_columns] = dataset[encoded_columns].astype(int)

# Rimuoviamo la colonna originale 'players_gender' che non serve più
dataset = dataset.drop(columns=['players_gender'])

# Riorganizziamo le colonne per mettere le colonne codificate all'inizio del dataset
encoded_columns = ['players_gender_M', 'players_gender_W', 'players_gender_X']
dataset = dataset[encoded_columns + [col for col in dataset.columns if col not in encoded_columns]]

# Visualizziamo il risultato per verificare che l'elaborazione sia corretta
dataset



,players_gender_M,players_gender_W,players_gender_X,event_id,match_id,match_format,is_final,final_set,opponent_final_set,stage_id,...,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,0,600,...,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,1,600,...,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,0,600,...,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,1,600,...,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0



##**Operazione su *single_matches_dataset***

Facciamo adesso alcune operazioni specifiche sulle feature su un dataset composto unicamente da partite di singolo

Filtrando solo i dati di singolo possiamo concentrarci su un **sottoinsieme specifico**, migliorando la qualità delle analisi.

In [27]:
# Rimuoviamo dal dataset le partite di doppio
rows_to_remove = dataset[dataset['match_format'] == 'D'].index

# Rimuoviamo le righe corrispondenti dal dataset originale
dataset_singles = dataset.drop(index=rows_to_remove)

# Rimuoviamo le colonne non desiderate poichè "players_gender_X" non può essere mai 1 perchè non ci sono doppi misti
columns_to_drop = ["players_gender_W", "players_gender_X"]
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

# Modificiamo il nome della tabella in "players_gender", 'M' = 1 e 'W' = 0
dataset_singles.rename(columns={"players_gender_M": "players_gender"}, inplace=True)

dataset_singles

,players_gender,event_id,match_id,match_format,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
16,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
17,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,113419,NaN,111683,NaN,2,3,"8-11,7-11,11-7,11-7,5-11","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...","[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...",0
18,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...",1
19,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,113419,NaN,111683,NaN,2,3,"8-11,7-11,11-7,11-7,5-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",0
20,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,1,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 7, 7, 8, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


## **Eliminazione delle colonne non adatte all'addestramento**

Vantaggi di questa operazione:
- **Eliminando colonne non necessarie**, il dataset diventa più semplice e focalizzato sui dati utili.
- Rimuovendo informazioni irrilevanti, l'addestramento risulta più chiaro e veloce, **evitando di appesantire** il modello con dati superflui.
- Con meno colonne da analizzare, è più **facile comprendere la struttura del dataset** e concentrarsi sugli aspetti importanti (ad esempio, il genere del giocatore o i set vinti).

In [28]:
# Definiamo le colonne da eliminare
columns_to_drop = ['event_id', 'match_id', 'stage_id', 'match_duration',
                  'match_start_time','opponent_id', 'opponent_2_id',
                  'player_sets_won', 'opponent_sets_won', 'match_scores',
                  'opponent_points','match_format', 'player_id', 'player_2_id']

# Rimuovo le colonne che non mi interessano
dataset = dataset.drop(columns=columns_to_drop, axis=1)
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

dataset_singles

,players_gender,is_final,final_set,opponent_final_set,player_points,set_result
16,1,1,0,0,"[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...",1
17,1,1,0,0,"[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",0
18,1,1,0,0,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",1
19,1,1,0,0,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...",0
20,1,1,1,0,"[1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, ...",0
...,...,...,...,...,...,...
144189,1,0,0,0,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,1,0,0,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",1
144191,1,0,1,0,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",0
144192,1,0,1,1,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


Salvataggio dei nuovi dataset

In [29]:
# Salviamo il dataset pulito dai valori indesiderati
dataset.to_csv("cleaned_dataset.csv", index=False)

In [30]:
# Salviamo il dataset finale delle partite di singolo
dataset_singles.to_csv("singles_matches_dataset.csv", index=False)